In [2]:
import sqlite3
import pandas as pd

In [3]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [4]:
data = pd.read_csv('boston.csv')

In [5]:
data.to_sql("boston", con, index=False, if_exists="replace")

In [6]:
df = pd.read_sql(
    """
    SELECT * FROM boston
    """,
    con,
)

In [7]:
df.sample(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
432,6.44405,0.0,18.10,0.0,0.5840,6.425,74.8,2.2004,24.0,666.0,20.2,97.95,12.03,16.1
282,0.06129,20.0,3.33,1.0,0.4429,7.645,49.7,5.2119,5.0,216.0,14.9,377.07,3.01,46.0
499,0.17783,0.0,9.69,0.0,0.5850,5.569,73.5,2.3999,6.0,391.0,19.2,395.77,15.10,17.5
48,0.25387,0.0,6.91,0.0,0.4480,5.399,95.3,5.8700,3.0,233.0,17.9,396.90,30.81,14.4
285,0.01096,55.0,2.25,0.0,0.3890,6.453,31.9,7.3073,1.0,300.0,15.3,394.72,8.23,22.0


### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [19]:
pd.read_sql(
    """
    SELECT
        sum(case when CRIM is NULL then 1 else 0 end) as crim_null,
        sum(case when ZN is NULL then 1 else 0 end) as zn_null,
        sum(case when INDUS is NULL then 1 else 0 end) as indus_null,
        sum(case when CHAS is NULL then 1 else 0 end) as chas_null,
        sum(case when NOX is NULL then 1 else 0 end) as nox_null
        
    FROM boston
    """,
    con,
)

,crim_null,zn_null,indus_null,chas_null,nox_null
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [28]:
pd.read_sql(
    """
    SELECT count(distinct crim) unique_crim,
           count(distinct zn) unique_zn,
           count(distinct indus) unique_indus,
           count(distinct chas) unique_chas,
           count(distinct nox) unique_nox
    
    FROM boston
    """,
    con,
)

,unique_crim,unique_zn,unique_indus,unique_chas,unique_nox
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [233]:
pd.read_sql(
    """
with
all_data as
(
    select 'crim' as key, crim as value from boston
    union all
    select 'zn' as key, zn as value from boston
    union all
    select 'chas' as key, chas as value from boston
    union all
    select 'indus' as key, indus as value from boston
    union all
    select 'nox' as key, nox as value from boston
),
stat_table as
(
    select key, value,
           row_number() over(partition by key order by value) as rn,
           min(value) over(partition by key) min_value,
           count(value) over(partition by key) as cnt
    from all_data
)
    

select key from (
                    select key, value as md_value, min_value
                    from stat_table
                    where rn = cnt/2
                )
where md_value=min_value
    
    
    """,
    con,
)
# как минимум первая половина значений, упорядоченных по возрастанию, в колонках chas и zn 
# равна минимальному значению столбца
# вариабельность значений в этих столбцах небольшая

,key
0,chas
1,zn


4. Выведите разницу между средним количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [144]:
pd.read_sql(
    """
    with avg_rm_max_medv as
    (
    SELECT avg(rm) avg_rm_max FROM boston
    where MEDV = (select max(MEDV) from boston)
    ), avg_tbl as
    (select avg(rm) over(order by medv rows between current row and 24 following) avg_25,
            avg(rm) over(order by medv rows between current row and 49 following) avg_50,
            avg(rm) over(order by medv rows between current row and 99 following) avg_100,
            avg(rm) over(order by medv rows between current row and 199 following) avg_200,
            avg(rm) over(order by medv rows between current row and 299 following) avg_300
     from boston
     limit 1
    )
    
    select '25' as cnt_houses, avg_25 as avg_rm, avg_rm_max, (avg_rm_max - avg_25) as diff    
    from avg_rm_max_medv, avg_tbl
    union all
    select '50' as cnt_houses, avg_50 as avg_rm, avg_rm_max, (avg_rm_max - avg_50) as diff    
    from avg_rm_max_medv, avg_tbl
    union all
    select '100' as cnt_houses, avg_100 as avg_rm, avg_rm_max, (avg_rm_max - avg_100) as diff    
    from avg_rm_max_medv, avg_tbl
    union all
    select '200' as cnt_houses, avg_200 as avg_rm, avg_rm_max, (avg_rm_max - avg_200) as diff    
    from avg_rm_max_medv, avg_tbl
    union all
    select '300' as cnt_houses, avg_300 as avg_rm, avg_rm_max, (avg_rm_max - avg_300) as diff    
    from avg_rm_max_medv, avg_tbl
    
    """,
    con,
)

# Количество комнат не сильно влияет на стоимость дома
# разница между 25 самыми дешевыми домами и 300 домами в среднем 0.23 комнаты
# но есть тенденция, что с увеличением стоимости домов, среднее количество комнат неизменно возрастает

,cnt_houses,avg_rm,avg_rm_max,diff
0,25,5.747840,7.484,1.736160
1,50,5.753240,7.484,1.730760
2,100,5.887120,7.484,1.596880
3,200,5.911705,7.484,1.572295
4,300,5.972227,7.484,1.511773


5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [228]:
pd.read_sql(
    """
    select lstat, medv, rank from 
    (select *, dense_rank() over(order by lstat) rank
    from boston)
    where MEDV = (select max(MEDV) from boston)
    """,
    con,
)

# в домах с самой высокой стоимостью небольшой процент населения с низким статусом
# в пределах первого квартиля

,LSTAT,MEDV,rank
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,14
6,3.26,50.0,15
7,3.32,50.0,16
8,3.70,50.0,23
9,3.73,50.0,24


6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [10]:
pd.read_sql(
    """
    SELECT case when chas=1 then 'граничит' else 'не граничит' end chas, avg(medv) 
    FROM boston
    group by chas
    
    """,
    con,
)

,chas,avg(medv)
0,не граничит,22.093843
1,граничит,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [ ]:
# выведите параметры, у которых среднее значение при условии,
# что дом граничит с рекой выше, чем среднее значение при обратном условии (CRIM, ZN, INDUS, NOX)

In [258]:
pd.read_sql(
    """
    with data as (
    select avg(case when chas=1 then crim end) avg_crim_1,
           avg(case when chas=0 then crim end) avg_crim_0,
           avg(case when chas=1 then zn end) avg_zn_1,
           avg(case when chas=0 then zn end) avg_zn_0,
           avg(case when chas=1 then indus end) avg_indus_1,
           avg(case when chas=0 then indus end) avg_indus_0,
           avg(case when chas=1 then nox end) avg_nox_1,
           avg(case when chas=0 then nox end) avg_nox_0
    from boston
    )
    
    select case when avg_crim_1>avg_crim_0 then 'crim'
                when avg_zn_1>avg_zn_0 then 'zn' 
                when avg_indus_1>avg_indus_0 then 'indus'
                when avg_nox_1>avg_nox_0 then 'nox'
                end name
    from data
    
    
    """,
    con,
)

# промышленных обьектов больше на территориях, граничащих с рекой

,name
0,indus


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [44]:
pd.read_sql(
    """
with data as(    
    SELECT *, ntile(10) over(order by nox) as nox_prc,
              ntile(10) over(order by indus) as indus_prc
    from boston)
    
    
select  t1.percentile, nox_value, indus_value
from 
       (select distinct nox_prc*10 as percentile, max(nox) over(partition by nox_prc) nox_value
        from data ) t1 join
        (select distinct indus_prc*10 as percentile, max(indus) over(partition by indus_prc) indus_value
        from data) t2
ON t1.percentile = t2.percentile
    """,
    con,
)

,percentile,nox_value,indus_value
0,10,0.426,2.89
1,20,0.442,4.39
2,30,0.472,5.96
3,40,0.507,7.38
4,50,0.538,9.69
5,60,0.580,13.89
6,70,0.609,18.10
7,80,0.671,18.10
8,90,0.713,19.58
9,100,0.871,27.74
